In [ ]:
import matplotlib.pyplot as plt

import os,sys
sys.path.append(r'{}'.format(os.path.abspath(os.pardir)))

from wholebrain_tools import aba, genes, dataIO
import wholebrain_tools.stats as st 
import wholebrain_tools.graphics as gt

# Instantiate an Atlas object from the aba module
# The first time you run this it will download the structures.json file from the Allen Institute server
paths = dataIO.pathParser()

nodes_path = paths.structures
A = aba.Atlas(nodes = nodes_path)
DFM = aba.AnatomyDataFrameManager(A)

# Load data

In [ ]:
# ------------------------------------------------------------------------------
searchpath =  paths.alldata
# ------------------------------------------------------------------------------

# Load dots data from all aniamls
df = dataIO.allMiceDots(searchpath)
df = DFM.multiIndexDf_from_dotsDf(df)

df.head()

# Colocalization of all areas in each subdivision

## Prepare data

In [ ]:
# Dataframe at mid ontology
mid_df = DFM.dotsRegionsDf_to_mid(df, numberOfCells=True)

# Perform the Leave-One-Out analysis at mid-ontology 
mid_df = DFM.dots_leaveOneOut_correction(
    mid_df,
    verbose=False,
    min_num_pnn=2,
    min_num_pv=2,
    min_num_mice=3,
    )

# Drop the dots in the fiber tracts
mid_df = mid_df.drop(1009, level='coarse')

mid_df

In [ ]:
temp = mid_df.groupby('params', axis=1).mean()
temp['acronym'] = A.ids_to_acronyms(temp.index.get_level_values('mid'))
temp = temp.droplevel('coarse')
temp

## All plots

In [ ]:
coarseAreas = mid_df.index.get_level_values('coarse').unique()

for coarseArea in coarseAreas:
    toPlot = mid_df.loc[coarseArea]
    gt.colocalizationDoubleBarplot(
        toPlot,
        A,
        x_left='pvPositive_pnn',
        x_right='wfaPositive_pv',
        xlabel_left='% of PNNs',
        xlabel_right='% of PV cells',
        title_left='PV$^+$ PNNs',
        title_right='WFA$^+$ PV cells',
        adaptiveHeight=True,
    )

    # plt.savefig(f"colocaliz_mid_{A.ids_to_acronyms([coarseArea])[0]}.svg", bbox_inches="tight")